In [32]:
import boto3
from json import dumps

In [29]:
bucket_name = 'cv-on-aws-book-tungps'
region_name = 'ap-southeast-1'
object_name = 'chapter_03/locate_packt_logo.jpeg'
rekognition = boto3.client('rekognition', region_name=region_name)

In [35]:
model='arn:aws:rekognition:ap-southeast-1:258831252650:project/Packt-logo-detection/version/Packt-logo-detection.2023-09-02T23.44.19/1693673059262'
min_confidence=0
response=rekognition.detect_custom_labels(
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': object_name
        }
    },
    MinConfidence=min_confidence,
    ProjectVersionArn=model
)
print(dumps(response, indent=2))

{
  "CustomLabels": [
    {
      "Name": "packt",
      "Confidence": 11.652999877929688,
      "Geometry": {
        "BoundingBox": {
          "Width": 0.0850600004196167,
          "Height": 0.13937999308109283,
          "Left": 0.8999300003051758,
          "Top": 0.07295999675989151
        }
      }
    },
    {
      "Name": "packt",
      "Confidence": 4.534999847412109,
      "Geometry": {
        "BoundingBox": {
          "Width": 0.1549600064754486,
          "Height": 0.16651000082492828,
          "Left": 0.5057299733161926,
          "Top": 0.38387998938560486
        }
      }
    },
    {
      "Name": "packt",
      "Confidence": 3.555999994277954,
      "Geometry": {
        "BoundingBox": {
          "Width": 0.14706000685691833,
          "Height": 0.17021000385284424,
          "Left": 0.34841999411582947,
          "Top": 0.3818899989128113
        }
      }
    },
    {
      "Name": "packt",
      "Confidence": 2.6000001430511475,
      "Geometry": {
        

In [37]:
# Load image from S3 bucket
s3_connection = boto3.resource('s3')

s3_object = s3_connection.Object(bucket_name,object_name)
s3_response = s3_object.get()

stream = io.BytesIO(s3_response['Body'].read())
image=Image.open(stream)

# Ready image to draw bounding boxes on it.
imgWidth, imgHeight = image.size
draw = ImageDraw.Draw(image)

# calculate and display bounding boxes for each detected custom label
print('Detected custom labels for ' + object_name)
for customLabel in response['CustomLabels']:
    print('Label \"' + str(customLabel['Name'])+'\"')
    print('Confidence ' + str(customLabel['Confidence']))
    if 'Geometry' in customLabel:
        box = customLabel['Geometry']['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']

        fnt = ImageFont.load_default()
        draw.text((left,top), customLabel['Name'], fill='#00d400', font=fnt)

        print('Left: ' + '{0:.0f}'.format(left))
        print('Top: ' + '{0:.0f}'.format(top))
        print('Label Width: ' + "{0:.0f}".format(width))
        print('Label Height: ' + "{0:.0f}".format(height))

        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top))
        draw.line(points, fill='#00d400', width=5)

image.show()

Detected custom labels for chapter_03/locate_packt_logo.jpeg
Label "packt"
Confidence 11.652999877929688
Left: 1350
Top: 22
Label Width: 128
Label Height: 42
Label "packt"
Confidence 4.534999847412109
Left: 759
Top: 115
Label Width: 232
Label Height: 50
Label "packt"
Confidence 3.555999994277954
Left: 523
Top: 115
Label Width: 221
Label Height: 51
Label "packt"
Confidence 2.6000001430511475
Left: 516
Top: 114
Label Width: 471
Label Height: 53
Label "packt"
Confidence 0.9510000348091125
Left: 1034
Top: 88
Label Width: 40
Label Height: 26
